In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
N_files = 10
run_number = 20079

In [3]:
N_centr = 5
N_sec = 2
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path=f"/media/yoren/T7 Shield/PHENIX/Dielectrons/fulls/pt_{run_number}/"
file_names=["my-411601.root"] 
sim_file_path="../sim/output/Newembed/sngl/"
legend_names = ["real","#pi^{0}","#eta","#gamma #rightarrow ee","c#bar{c}","b#bar{b}"] 
hist_select_3D_names = ["stof_hist","ttof","myvtx_hist_2",'myvtx_hist_3','el_pt_hist_0','el_pt_hist_5','event_hist']

In [4]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [5]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


infile.Close()

/media/yoren/T7 Shield/PHENIX/Dielectrons/fulls/pt_20079/my-411601.root


In [6]:
type_loc = 4
pt = [2.5,4.5]
c0 = root.TCanvas(f"c0",f"c0",700,700)
c0.Divide(1,1)
legends = []
proj_y = hists_read[0][type_loc].ProjectionY(hists_read[0][type_loc].GetName()+f"pos_proj_y",1,-1,1,-1)
biny0 = proj_y.FindBin(pt[0])
biny1 = proj_y.FindBin(pt[1])
for isec in range(1):
    c0.cd(1+isec)
    ibinz0 = 10
    ibinz1 = 25
    pos_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"pos_proj_{isec}",biny0,biny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="x, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.Draw()
    #pos_proj.Fit("gaus","R","")
    legends.append(root.TLegend(0.5,0.6,0.95,0.95,f"Sector: {isec}, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(1),pos_proj.FindBin(2)),hists_read[0][-1].GetBinContent(6))

    print(f"integral: {pos_proj.GetMean()*10000:.0f}, {pos_proj.GetStdDev()*10000:.0f} mum")
    var1 = pos_proj.Integral(pos_proj.FindBin(1),pos_proj.FindBin(2))/hists_read[0][-1].GetBinContent(6)*1e3
    print(f"var1: {var1:.3f}")

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_sec.png")

13631.0 16255494.0
integral: 6128, 2958 mum
var1: 0.839


python ERROR: cannot open image file "output/rich_emcal_qa/qa_pt_sec.png" for writing. Please check permissions.


reading file list

In [7]:
import os
import re

file_pattern = re.compile(r"my-(\d{6})\.root")

file_numbers = []

for filename in os.listdir(file_path):
    match = file_pattern.match(filename)
    if match:
        file_numbers.append(match.group(1))

print(len(file_numbers))

1071


In [8]:
file_numbers.sort()

In [9]:
#file_numbers= file_numbers[:N_files]

In [10]:
pt = [2.5,4.5]
biny0 = proj_y.FindBin(pt[0])
biny1 = proj_y.FindBin(pt[1])
print(biny0, biny1)

3 5


In [11]:
def vertex_estimator(file_number, N_hists_local = [0]):
    infile = root.TFile.Open(file_path+f"my-{file_number}.root", "read")
    hists_read_file = []
    for ihist in N_hists_local:
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{file_number}")
    hitst_stat = infile.Get("event_hist")
    hitst_stat.SetDirectory(root.nullptr)

    fits = []
    params_to_return = []
    N_sec = len(N_hists_local)
    for isec in range(N_sec):
        c1 = root.TCanvas(f"c1_{file_number}_{isec}", f"c1_{file_number}_{isec}", 600, 600)
        ibin1 = 25; ibin0 = 10
        pos_projX = hists_read_file[isec].ProjectionX(hists_read_file[isec].GetName()+f"pos_projX_{file_number}_{isec}",biny0,biny1,ibin0,ibin1)
        integral = pos_projX.Integral(pos_projX.FindBin(1), pos_projX.FindBin(2))
        entries = hitst_stat.GetBinContent(6)
        Y = integral/entries*1e3 if entries>0 else 0
        Y_err = Y**0.5/entries*1e3 if entries>0 else 0
        params_to_return.append((Y,Y_err, integral, entries)) 

    return params_to_return



In [12]:
#file_numbers = file_numbers[:10]
#print(file_numbers)

In [13]:
means, sigmas, mean_errs, sigma_errs, entries = [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)]
for file_number in file_numbers:
    params_for_all_secotrs = vertex_estimator(file_number, N_hists_local = [4,5])
    for isec in range(N_sec):
        mean, mean_err, sigma, entries_local = params_for_all_secotrs[isec]
        means[isec].append(mean)
        sigmas[isec].append(sigma)
        mean_errs[isec].append(mean_err) 
        entries[isec].append(entries_local)

In [14]:
# Create canvases
c1 = root.TCanvas("c1", "Mean  vs File Numbers", 1200,600)

# Convert file_numbers to integers for plotting
file_numbers_int = [int(num) for num in file_numbers]

# Create graphs

# Customize and draw graphs
def customize_and_draw(iOpt, N_sectors, canvas, title, x_title, y_title):

    canvas.Divide(2,1)
    graphs = []
    for isec in range(N_sectors):
        canvas.cd(isec+1)
        graph =  root.TGraphErrors(len(file_numbers), array('d', file_numbers_int),  array('d', means[isec]), 0, array('d', mean_errs[isec]))
        if iOpt: graph = root.TGraphErrors(len(file_numbers), array('d', file_numbers_int), array('d', sigmas[isec]), 0, array('d', sigma_errs[isec]))
        graph.SetTitle(title)
        Format_Hist_total(graph, Msize=1, Mcolor=4, Mstyle=20, title_x=x_title, title_y=y_title, left=0.17, offset_y=1.1)
        graph.Draw("AP")
        graph.GetYaxis().SetRangeUser(0.,1.2)
        canvas.Draw()
        graphs.append(graph)

    return graphs

results = []
results.append(customize_and_draw(0,2, c1, "Mean vs File Numbers", "File Numbers", "Mean"))

In [15]:
good_runs = []; bad_runs = []
for i in range(len(means[0])):
    if 0.7 < means[0][i] < 1.2 or 0.7 < means[1][i] < 1.2 :
        good_runs.append(file_numbers[i])
    else:
        bad_runs.append(file_numbers[i])

In [16]:
print(len(good_runs), len(bad_runs))

960 111


In [17]:
print("Good runs:", good_runs)
print("Bad runs:", bad_runs)

Good runs: ['405863', '405864', '405865', '405868', '405869', '405961', '405964', '405966', '405971', '405972', '405973', '405975', '405995', '405996', '406088', '406089', '406090', '406093', '406094', '406095', '406099', '406180', '406182', '406183', '406190', '406257', '406258', '406259', '406261', '406263', '406265', '406266', '406268', '406543', '406544', '406546', '406549', '406555', '406556', '406579', '406581', '406582', '406584', '406661', '406662', '406663', '406666', '406671', '406674', '406675', '406676', '406677', '406696', '406697', '406698', '406700', '406745', '406746', '406747', '406748', '406753', '406754', '406759', '406760', '406761', '406762', '406764', '406766', '406767', '406769', '406772', '406773', '406774', '406830', '406831', '406832', '406833', '406848', '406849', '406850', '406851', '406853', '406857', '406858', '406859', '406861', '406866', '406867', '406869', '406870', '406872', '406881', '406882', '406889', '406891', '406902', '406905', '406920', '406921'

In [18]:
print(f"hadd -f -k -O -j 16 -n 10000 ../m_ee_Run14AuAu_157th_new_{run_number}_1071runs_goodruns1.root " + " ".join([f"{run}.root " for run in good_runs]))

hadd -f -k -O -j 16 -n 10000 ../m_ee_Run14AuAu_157th_new_20079_1071runs_goodruns1.root 405863.root  405864.root  405865.root  405868.root  405869.root  405961.root  405964.root  405966.root  405971.root  405972.root  405973.root  405975.root  405995.root  405996.root  406088.root  406089.root  406090.root  406093.root  406094.root  406095.root  406099.root  406180.root  406182.root  406183.root  406190.root  406257.root  406258.root  406259.root  406261.root  406263.root  406265.root  406266.root  406268.root  406543.root  406544.root  406546.root  406549.root  406555.root  406556.root  406579.root  406581.root  406582.root  406584.root  406661.root  406662.root  406663.root  406666.root  406671.root  406674.root  406675.root  406676.root  406677.root  406696.root  406697.root  406698.root  406700.root  406745.root  406746.root  406747.root  406748.root  406753.root  406754.root  406759.root  406760.root  406761.root  406762.root  406764.root  406766.root  406767.root  406769.root  406